<a href="https://colab.research.google.com/github/sberniz/DS-Unit-2-Linear-Models/blob/master/module2-regression-2/LS_DS_212_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [47]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score #import libraries
from sklearn.linear_model import LinearRegression
# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [48]:
#Engineer New Features at least two 
df['total_rooms'] = df['bathrooms'] + df['bedrooms']
df['ratio_bed_bath'] = df['bedrooms'] / df['bathrooms']

In [49]:
#Change object to date created 
df['created'] = pd.to_datetime(df['created'],infer_datetime_format=True)


In [50]:
#Create X and y and Split Data
X = df.drop('price', axis=1)
y = df['price']
X_train = X[(X['created'] < '2016-06-01') & (X['created'] >='2016-04-01')] #Date created before June and after May
X_test = X[X['created'] >= '2016-06-01'] #Date from june 1 and ahead
y_train = y[y.index.isin(X_train.index)] #Pairs Indexes of X_train with Y_train
y_test = y[y.index.isin(X_test.index)] #y test pair to be same as  X_test
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(31844, 35)
(16973, 35)
(31844,)
(16973,)


In [51]:
#Mean Baseline
y_train.mean()

3575.604007034292

In [55]:
#Train error on base model without any linear regression 
y_pred_base = [y_train.mean()] * len(y_train)
mae = mean_absolute_error(y_train,y_pred_base)
print('Train Error for Baseline mean',mae)

Train Error for Baseline mean 1201.8811133682555


In [34]:
#Fit linear regression with two features


In [75]:
X_train.head(1)

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,total_rooms,ratio_bed_bath
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,1.0


In [53]:
#Fit Lenear Regrassion with two features
features = ['bathrooms','bedrooms']
model_2_feat = LinearRegression()
model_2_feat.fit(X_train[features],y_train)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [41]:
#model coeeficient and intercepts
print('Intercept',model_2_feat.intercept_)
coefficients = pd.Series(model_2_feat.coef_,features)
print(coefficients.to_string())

Intercept 485.71869002322865
bathrooms    2072.610116
bedrooms      389.324896


In [42]:
model_2_feat.coef_

array([2072.61011639,  389.3248959 ])

In [60]:
y_pred2_train = model_2_feat.predict(X_train[features])
y_pred2_test = model_2_feat.predict(X_test[features])
y_pred2_test

array([4762.60855231, 3336.97859821, 2947.65370231, ..., 2947.65370231,
       3336.97859821, 3336.97859821])

In [72]:
#Get regression metrics for RMSE, MAE, and R2
#Mean Absolute Error 

mae_train = mean_absolute_error(y_train,y_pred2_train)
print('Training Mean Absolute Error',mae_train)
mae_test = mean_absolute_error(y_test,y_pred2_test)
print('Testing Mean Absolute Error',mae_test)
#root mean square error
mserror = mean_squared_error(y_train,y_pred2_train)
print('rmse is:',np.sqrt(mserror))
#Test rmse 
mseerrortest = mean_squared_error(y_test,y_pred2_test)
print('rmse_test is',np.sqrt(mseerrortest))

Training Mean Absolute Error 818.5310213271714
Testing Mean Absolute Error 825.8987822403527
rmse is: 1232.0225917223484
rmse_test is 1219.719357233823


In [76]:
#5 feature test
features5 = ['dogs_allowed','cats_allowed','bathrooms','bedrooms','roof_deck']
model_5_feat = LinearRegression()
model_5_feat.fit(X_train[features5],y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [80]:
y_5_pred = model_5_feat.predict(X_train[features5])
mae5_train = mean_absolute_error(y_train,y_5_pred)
y_5_pred_test = model_5_feat.predict(X_test[features5])
print('MAE Training',mae5_train)
mae5_test = mean_absolute_error(y_test,y_5_pred_test)
print('Mae Test',mae5_test)

MAE Training 811.9308183080643
Mae Test 818.8464273493674


In [108]:
model_5_feat.intercept_, model_5_feat.coef_

(410.68671531135715,
 array([ 257.28091456,  -97.72979076, 2043.279824  ,  392.57121176,
         278.78623821]))